<a href="https://colab.research.google.com/github/nabilakartika/RecomendationMovie/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMBIL DATA DARI KAGGLE

In [ ]:
from google.colab import files
files.upload()

: 

In [ ]:
# Konfigurasi
import os
import shutil

# Buat direktori .kaggle di home
os.makedirs('/root/.kaggle', exist_ok=True)

# Pindahkan file kaggle.json ke direktori .kaggle
shutil.move('kaggle (3).json', '/root/.kaggle/kaggle.json')

# Ubah izin untuk file kaggle.json
os.chmod('/root/.kaggle/kaggle.json', 600)

: 

In [ ]:
#Download dataset
!kaggle datasets download -d rahulvyasm/netflix-movies-and-tv-shows

: 

In [ ]:
import zipfile

# Ekstrak file zip
with zipfile.ZipFile('/content/netflix-movies-and-tv-shows.zip', 'r') as zip_ref:
    zip_ref.extractall('netflix-movies-and-tv-shows')

: 

In [ ]:
import pandas as pd

df = pd.read_csv('/content/netflix-movies-and-tv-shows/netflix_titles.csv', encoding='ISO-8859-1')
df.tail(20)

: 

In [ ]:
df.shape

: 

In [ ]:
df.describe()

: 

In [ ]:
df.info()

: 

In [ ]:
df.drop(columns=['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
                   'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
                   'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
                   'Unnamed: 24', 'Unnamed: 25'], inplace=True)

: 

In [ ]:
df.info()

: 

In [ ]:
df.isna().sum()

: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
df.to_csv('/content/drive/MyDrive/DibimbingFP/netflix_edited.csv', index=False)

: 

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/DibimbingFP/netflix_edited.csv')
df1

: 

In [ ]:
df1['director'].fillna('Unknown', inplace=True)
df1['cast'].fillna('Unknown', inplace=True)
df1['country'].fillna('Unknown', inplace=True)

: 

In [ ]:
df1

: 

In [ ]:
df1.to_csv('/content/drive/MyDrive/DibimbingFP/newdata_netflix.csv', index=False)

: 

# MELIHAT PERKEMBANGAN FILM YANG DIRILIS DARI TAHUN KE TAHUN

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

# Menghitung skewness untuk kolom 'release_year'
release_year_skewness = skew(df1['release_year'].dropna())
print(f"Skewness of release_year: {release_year_skewness}")

plt.figure(figsize=(10, 6))
sns.histplot(df1['release_year'].dropna(), kde=True)
plt.title('Distribution of Release Year')
plt.xlabel('Release Year')
plt.ylabel('Frequency')
plt.show()

: 

In [ ]:
# Menghitung jumlah rilis per genre setiap tahun
# Convert 'listed_in' column to string type
df1['listed_in'] = df1['listed_in'].astype(str)

# Now split the genre
df1['listed_in'] = df1['listed_in'].str.split(', ')

# Membuat kolom genre
df_exploded = df1.explode('listed_in')
genre_year_count = df_exploded.groupby(['release_year', 'listed_in']).size().reset_index(name='count')
genre_year_sorted = genre_year_count.sort_values(by=['release_year', 'count'], ascending=[True, False])

print(genre_year_sorted)

: 

In [ ]:
# Visualisasi: Menampilkan tren genre dari tahun ke tahun
plt.figure(figsize=(14, 8))
sns.lineplot(data=genre_year_sorted, x='release_year', y='count', hue='listed_in')
plt.title('Jumlah Rilis Film per Genre Tiap Tahun')
plt.xlabel('Tahun Rilis')
plt.ylabel('Jumlah Rilis')
plt.legend(title='Genre', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the figure size
plt.figure(figsize=(14, 8))

# Create the bar plot
sns.barplot(data=genre_year_sorted, x='listed_in', y='count', hue='release_year', dodge=False)

# Set the title and labels
plt.title('Jumlah Rilis Film per Genre')
plt.xlabel('Genre')
plt.ylabel('Jumlah Rilis')
plt.xticks(rotation=90)
plt.legend(title='Tahun', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

: 

In [ ]:
# Mengetahui 20 genre yang paling banyak dirilis
genre_counts = genre_year_sorted.groupby('listed_in')['count'].sum()
top_20_genres = genre_counts.nlargest(20)

print(top_20_genres)

: 

In [ ]:
# Mengetahui genre yang paling banyak dirilis berdasarkan tahun
genre_year_counts = genre_year_sorted.groupby(['listed_in', 'release_year'])['count'].sum()
most_released_genre_year = genre_year_counts.idxmax()
genre_terbanyak, tahun_terbanyak = most_released_genre_year
jumlah_rilis_terbanyak = genre_year_counts.max()

print("Genre yang paling banyak dirilis:", genre_terbanyak)
print("Tahun:", tahun_terbanyak)
print("Jumlah rilis:", jumlah_rilis_terbanyak)


: 

In [ ]:
# MANA YANG LEBIH BANYAK DIRILIS? MOVIE or TV SHOW?
size = df1['type'].value_counts()
labels = 'Movies', 'TV show'
color = sns.color_palette('Paired')

plt.pie(size, colors=color, labels=labels, autopct='%1.2f%%')
plt.title('Movies vs TV shows')
plt.show()

: 

In [ ]:
movies = df1[df1['type'] == 'Movie']
tvshows = df1[df1['type'] == 'TV Show']

: 

**NEGARA MANA YANG MERILIS MOVIE/TV SHOW PALING BANYAK??**

In [ ]:
def count_productions(df1):
    country_series = df1['country'].dropna().str.split(', ').explode()
    return country_series.value_counts()

# Movie
movies_counts = count_productions(movies)
print("Countries with the most films produced on Netflix:")
print(movies_counts.head(10), "\n")

# TV Shows
tvshows_counts = count_productions(tvshows)
print("Countries with the most TV shows produced on Netflix:")
print(tvshows_counts.head(10), "\n")

# Total produksi per negara
total_counts = count_productions(df1)
print("Countries with the most products on Netflix:")
print(total_counts.head(10), "\n")


: 

**MOST POPULAR DIRECTOR**

In [ ]:
def count_directors(df1):
    director_series = df1['director'].dropna().str.split(', ').explode()
    return director_series.value_counts()

# Movie
movie_directors = count_directors(movies)
print("Most popular movie directors on Netflix:")
print(movie_directors.head(10), "\n")

# TV Show
tvshow_directors = count_directors(tvshows)
print("Most popular TV show directors on Netflix:")
print(tvshow_directors.head(10), "\n")

: 

**PERILISAN TERBANYAK BERDASARKAN TAHUN**

In [ ]:
def count_by_year(df1):
    year_added = df1['date_added'].dropna().apply(lambda x: x[-4:])
    return year_added.value_counts()

# Movie
myear = count_by_year(movies)
print("Years with the most movies added to Netflix:")
print(myear.head(), "\n")

# TV Show
tvyear = count_by_year(tvshows)
print("Years with the most TV shows added to Netflix:")
print(tvyear.head())


: 

In [ ]:
# Fungsi untuk menghitung jumlah penambahan film dan acara TV per tahun
def count_by_year(df):
    year_added = df['date_added'].dropna().apply(lambda x: x[-4:])
    return year_added.value_counts()

# Filter data untuk film
movies_by_year = count_by_year(movies)
print("Years with the most movies added to Netflix:")
print(movies_by_year.head(1))  # Tampilkan tahun dengan jumlah penambahan film terbanyak

# Filter data untuk acara TV
tvshows_by_year = count_by_year(tvshows)
print("\nYears with the most TV shows added to Netflix:")
print(tvshows_by_year.head(1))  # Tampilkan tahun dengan jumlah penambahan acara TV terbanyak


: 

**DURASI**

In [ ]:
def count_by_year(df1):
    year_added = df1['date_added'].dropna().apply(lambda x: x[-4:])
    return year_added.value_counts()

movies_byyear = count_by_year(movies)
tvshows_byyear = count_by_year(tvshows)

# Plot untuk film
plt.figure(figsize=(10, 6))
movies_by_year.plot(kind='bar', color='grey')
plt.title('Movies Added to Netflix by Year')
plt.xlabel('Year')
plt.ylabel('Number of Movies')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Plot untuk acara TV
plt.figure(figsize=(10, 6))
tvshows_by_year.plot(kind='bar', color='brown')
plt.title('TV Shows Added to Netflix by Year')
plt.xlabel('Year')
plt.ylabel('Number of TV Shows')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()


: 

In [ ]:
# MOVIE DURATIONS
movie = df1[df1['type'] == 'Movie']

# Ekstrak dalam menit
movie_duration = movie['duration'].str.extract('(\d+)').astype(float)

plt.figure(figsize=(12, 6))
sns.histplot(data=movie_duration, kde=True, color='skyblue', edgecolor='black')
plt.title('Distribution of Movie Durations on Netflix')
plt.xlabel('Duration (minutes)')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

: 

In [ ]:
# TV Show season
tvshows_df = df1[df1['type'] == 'TV Show']

tvshow_seasons = tvshows_df['duration'].value_counts().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=tvshow_seasons.index, y=tvshow_seasons.values, color='green')
plt.title('Distribution of TV Show Seasons on Netflix')
plt.xlabel('Number of Seasons')
plt.xticks(rotation=45)
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

: 

**RATING**

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(data=df1, x='rating', order=df1['rating'].value_counts().index)
plt.title('Rating on Netflix')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


: 

# DNN

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

: 

In [ ]:
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Input, Reshape, Dot, Embedding
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import Concatenate, Dense, Dropout
from keras.layers import Add, Activation, Lambda


import tensorflow as tf
import keras
from pprint import pprint

: 

In [ ]:
df_netflix = pd.read_csv('/content/drive/MyDrive/DibimbingFP/newdata_netflix.csv')
df_netflix

: 

In [ ]:
# Membuat userId
np.random.seed(42)
num_users = 8908
df_netflix['userId'] = np.random.randint(1, num_users + 1, df_netflix.shape[0])

: 

In [ ]:
# Membuat movieId
df_netflix['movieId'] = df_netflix['show_id'].astype('category').cat.codes

: 

Karena rating dari data tidak bisa digunakan, maka dibuat sendiri colom rating_new dalam bentuk angka sehingga dapat digunakan sebagai rekomendasi

In [ ]:
df_netflix['rating_new'] = np.random.randint(1, 6, df_netflix.shape[0])

: 

In [ ]:
movie_dataset = df_netflix[['show_id','userId','movieId','title','rating_new']]
movie_dataset.head()

: 

In [ ]:
num_users = len(df_netflix['userId'].value_counts())
num_items = len(df_netflix['title'].value_counts())
print('Unique number of users in the dataset: {}'.format(num_users))
print('Unique number of movies in the dataset: {}'.format(num_items))

: 

In [ ]:
rating_count_df = pd.DataFrame(df_netflix.groupby(['rating_new']).size(), columns=['count'])
rating_count_df

: 

Membuat dataset berdasarkan userID, judul movie, dan juga rating

In [ ]:
refined_dataset = movie_dataset.groupby(by=['userId','title'], as_index=False).agg({"rating_new":"mean"})
refined_dataset.head()

: 

**Encoding users and movie titles to make sure that the sequence has no missing values when dealing with Deep Neural Networks.**

In [ ]:
user_enc = LabelEncoder()
refined_dataset['user'] = user_enc.fit_transform(refined_dataset['userId'].values)
n_users = refined_dataset['user'].nunique()

: 

In [ ]:
item_enc = LabelEncoder()
refined_dataset['movie'] = item_enc.fit_transform(refined_dataset['title'].values)
n_movies = refined_dataset['movie'].nunique()

: 

In [ ]:
refined_dataset['rating'] = refined_dataset['rating_new'].values.astype(np.float32)
min_rating = min(refined_dataset['rating'])
max_rating = max(refined_dataset['rating'])
n_users, n_movies, min_rating, max_rating

: 

In [ ]:
refined_dataset.head()

: 

Splitting the data into training and testing

In [ ]:
X = refined_dataset[['user', 'movie']].values
y = refined_dataset['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

: 

In [ ]:
n_factors = 100

: 

In [ ]:
X_train

: 

In [ ]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

: 

In [ ]:
X_train, X_train_array, X_train_array[0].shape

: 

In [ ]:
import numpy as np
np.array(X_train_array).shape

: 

In [ ]:
y_train = (y_train - min_rating)/(max_rating - min_rating)
y_test = (y_test - min_rating)/(max_rating - min_rating)

: 

In [ ]:
import numpy as np

# Ensure X_train_array and X_test_array are NumPy arrays
X_train_array = np.array(X_train_array)
X_test_array = np.array(X_test_array)

# Split the arrays into user IDs and movie IDs
X_train_users = X_train_array[0, :]
X_train_movies = X_train_array[1, :]

X_test_users = X_test_array[0, :]
X_test_movies = X_test_array[1, :]
X_train_users.shape

: 

In [ ]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot, Embedding, Concatenate, Dense, Dropout, Activation, Lambda
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
import tensorflow as tf

# Define the number of users and movies, and the number of latent factors
n_users = 1000   # Replace with the actual number of users
n_movies = 500   # Replace with the actual number of movies
n_factors = 50   # Number of latent factors

# Initializing an input layer for users
user = Input(shape=(1,))

# Embedding layer for n_factors of users
u = Embedding(n_users, n_factors, embeddings_initializer='he_normal', embeddings_regularizer=l2(1e-6))(user)
u = Reshape((n_factors,))(u)

# Initializing an input layer for movies
movie = Input(shape=(1,))

# Embedding layer for n_factors of movies
m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal', embeddings_regularizer=l2(1e-6))(movie)
m = Reshape((n_factors,))(m)

# Stacking up both user and movie embeddings
x = Concatenate()([u, m])
x = Dropout(0.05)(x)

# Adding Dense layers to the architecture
x = Dense(32, kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Dense(16, kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

# Adding an Output layer with Softmax activation function
x = Dense(10)(x)  # Assuming the rating range is from 0 to 9
x = Activation('softmax')(x)

# Defining the model
model = Model(inputs=[user, movie], outputs=x)

# Compiling the model
optimizer = SGD(lr=0.001)  # You can choose other optimizers like Adam if needed
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

: 

In [ ]:
import numpy as np

# Check the maximum values of user and movie IDs in your data
max_user_id = max(max(X_train_array[0, :]), max(X_test_array[0, :]))
max_movie_id = max(max(X_train_array[1, :]), max(X_test_array[1, :]))

# Update n_users and n_movies accordingly
n_users = max_user_id + 1
n_movies = max_movie_id + 1

print(f"Number of users: {n_users}")
print(f"Number of movies: {n_movies}")

# Reinitialize the model with the updated n_users and n_movies
user = Input(shape=(1,))
u = Embedding(n_users, n_factors, embeddings_initializer='he_normal', embeddings_regularizer=l2(1e-6))(user)
u = Reshape((n_factors,))(u)

movie = Input(shape=(1,))
m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal', embeddings_regularizer=l2(1e-6))(movie)
m = Reshape((n_factors,))(m)

x = Concatenate()([u, m])
x = Dropout(0.05)(x)

x = Dense(32, kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Dense(16, kernel_initializer='he_normal')(x)
x = Activation('relu')(x)
x = Dropout(0.05)(x)

x = Dense(10)(x)
x = Activation('softmax')(x)

model = Model(inputs=[user, movie], outputs=x)
optimizer = SGD(lr=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

: 

In [ ]:
# Verify the shapes after splitting
print(f"Shape of X_train_users: {X_train_users.shape}")
print(f"Shape of X_train_movies: {X_train_movies.shape}")
print(f"Shape of X_test_users: {X_test_users.shape}")
print(f"Shape of X_test_movies: {X_test_movies.shape}")

# Assuming all checks pass, we proceed with the training
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=3, min_lr=0.000001, verbose=1)

history = model.fit(
    x=[X_train_users, X_train_movies],
    y=y_train,
    batch_size=128,
    epochs=70,
    verbose=1,
    validation_data=([X_test_users, X_test_movies], y_test),
    shuffle=True,
    callbacks=[reduce_lr]
)

: 

In [ ]:
plt.plot(history.history["loss"][5:])
plt.plot(history.history["val_loss"][5:])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

: 

In [ ]:
refined_dataset.head()

: 

In [ ]:
X_train_array

: 

In [ ]:
user_id = [975]
encoded_user_id = user_enc.transform(user_id)

seen_movies = list(refined_dataset[refined_dataset['userId'] == user_id[0]]['movie'])
print(seen_movies)

: 

In [ ]:
len(refined_dataset['movie'].unique()), min(refined_dataset['movie']), max(refined_dataset['movie'])

: 

In [ ]:
unseen_movies = [i for i in range(min(refined_dataset['movie']), max(refined_dataset['movie'])+1) if i not in seen_movies]
print(unseen_movies)

: 

In [ ]:
len(unseen_movies) + len(seen_movies)

: 

In [ ]:
model_input = [np.asarray(list(encoded_user_id)*len(unseen_movies)), np.asarray(unseen_movies)]
len(model_input), len(model_input[0])

: 

In [ ]:
model.save('/content/drive/MyDrive/DibimbingFP/NetflixRecomendation.h5')

: 

DNN model is used to predict the ratings of the unseen movies

In [ ]:
predicted_ratings = model.predict(model_input)

: 

In [ ]:
print(predicted_ratings.shape)

: 

In [ ]:
print(predicted_ratings)

: 

In [ ]:
predicted_ratings = np.max(predicted_ratings, axis=1)
predicted_ratings

: 

In [ ]:
predicted_ratings.shape

: 

In [ ]:
sorted_index = np.argsort(predicted_ratings)[::-1]
print(sorted_index)

: 

Movie names have been extracted from the available indices we got

In [ ]:
recommended_movies = item_enc.inverse_transform(sorted_index)
recommended_movies

: 

Movies recommended with the help of Softmax Deep Neural Networks

In [ ]:
from pprint import pprint
pprint(list(recommended_movies[:20]))

: 

Summing up the entire code into a recommender system function:

In [ ]:
def recommender_system(user_id, model, n_movies):

  print("")
  print("Movie seen by the User:")
  pprint(list(refined_dataset[refined_dataset['userId'] == user_id]['title']))
  print("")

  encoded_user_id = user_enc.transform([user_id])

  seen_movies = list(refined_dataset[refined_dataset['userId'] == user_id]['movie'])
  unseen_movies = [i for i in range(min(refined_dataset['movie']), max(refined_dataset['movie'])+1) if i not in seen_movies]
  model_input = [np.asarray(list(encoded_user_id)*len(unseen_movies)), np.asarray(unseen_movies)]
  predicted_ratings = model.predict(model_input)
  predicted_ratings = np.max(predicted_ratings, axis=1)
  sorted_index = np.argsort(predicted_ratings)[::-1]
  recommended_movies = item_enc.inverse_transform(sorted_index)
  print("---------------------------------------------------------------------------------")
  print("Top "+str(n_movies)+" Movie recommendations for the User "+str(user_id)+ " are:")
  pprint(list(recommended_movies[:n_movies]))

: 

In [ ]:
print("Enter user id")
user_id= int(input())

print("Enter number of movies to be recommended:")
n_movies = int(input())
recommender_system(user_id,model,n_movies)


: 

Rough Work

In [ ]:
a = np.asarray([[2,3,14,6],[3,5,7,2],[6,8,4,1]])
a, a.shape

: 

In [ ]:
b=np.max(a, axis=1)
b, b.shape

: 

In [ ]:
np.argsort(b)[::-1]

: 

In [ ]:
user_enc.transform([10])

: 

In [ ]:
max(refined_dataset['user'])

: 

: 